In [78]:
import time
import random
import math

In [2]:
people = [('Seymour', 'BOS'),
          ('Franny', 'DAL'),
          ('Zooey', 'CAK'),
          ('Walt', 'MIA'),
          ('Buddy', 'ORD'),
          ('Les', 'OMA')]

# LaGuardia airport in New York.
destination = 'LGA'

In [5]:
flights = {}
with open('schedule.txt') as f:
    for line in f:
        origin, dest, depart, arrive, price = line.strip().split(',')
        flights.setdefault((origin, dest), [])
        
        # Add details to the list of possible flights.
        flights[(origin,dest)].append((depart,arrive,int(price)))

list(flights.keys())[:5]

[('LGA', 'OMA'),
 ('OMA', 'LGA'),
 ('LGA', 'ORD'),
 ('ORD', 'LGA'),
 ('LGA', 'MIA')]

In [81]:
import time

def get_minutes(t):
    x = time.strptime(t, '%H:%M')
    hr, mi = x[3], x[4]
    return hr * 60 + mi

In [82]:
def print_schedule(r):
    for d in range(len(r)//2):
        (name, origin) = people[d]
        out = flights[(origin,destination)][r[d]]
        ret = flights[(destination,origin)][r[d+1]]
        print(f'{name:10s}{origin:10s} {out[0]:>5s}-{out[1]:>5s} ${str(out[2]):>3s} {ret[0]:>5s}-{ret[1]:>5s} ${str(ret[2]):>3s}')

In [83]:
# An array pair of origin and destination.
s = [1,4,3,2,7,3,6,3,2,4,5,3]
print_schedule(s)

Seymour   BOS         8:04-10:11 $ 95 12:08-14:05 $142
Franny    DAL        12:19-15:25 $342 10:51-14:16 $256
Zooey     CAK        10:53-13:36 $189  9:58-12:56 $249
Walt      MIA         9:15-12:29 $225 16:50-19:26 $304
Buddy     ORD        16:43-19:00 $246 10:33-13:11 $132
Les       OMA        11:08-13:07 $175 15:07-17:21 $129


## Cost Function

The goal of any optimization algorithm is to find a set of inputs that minimizes the cost function, so the cost function has to return a value that represents how bad a solution is. The function should return larger values for worse solutions.

In the scenario for flights:
- price: the total price of all plane tickets
- travel time: the time everyone has to spend on a plane
- departure time: flight that leave too erly in the morning may impose an additional cost by requiring travelers to miss out on sleep
- car rental period: if the party rents a car, they must return it earlier in the day than when they rented it, or be forced to pay for a whole extra day

In [84]:
def schedule_cost(solution):
    if solution is None: return -1
    total_price = 0
    latest_arrival = 0
    earliest_departure = 24 * 60
    
    for d in range(len(solution) // 2):
        # Get the inbound and outbound flights.
        (_,origin) = people[d]
        outbound = flights[(origin,destination)][int(solution[d])]
        returnf = flights[(destination,origin)][int(solution[d+1])]
        
        # Destructure to simplify. o_: outbound, r_: return.
        (o_depart, o_arrive, o_price) = outbound
        (r_depart, r_arrive, r_price) = returnf
        
        # Total price is the price of all outbound and return flight.
        total_price += o_price
        total_price += r_price
        
        # Track the latest arrival and earliest departure.
        if latest_arrival < get_minutes(o_arrive): latest_arrival = get_minutes(o_arrive)
        if earliest_departure > get_minutes(r_depart): earliest_departure = get_minutes(r_depart)
        
    # Every person must wait at the airport until the latest person arrives.
    # They also must arrive at the same time and wait for their flights.
    total_wait = 0
    for d in range(len(solution) // 2):
        (_,origin) = people[d]
        outbound = flights[(origin,destination)][int(solution[d])]
        returnf = flights[(destination,origin)][int(solution[d+1])]
        (_,o_arrive,_) = outbound
        (r_depart,_,_) = returnf
        total_wait += latest_arrival - get_minutes(o_arrive)
        total_wait += get_minutes(r_depart) - earliest_departure
    
    # Does this solution require an exra day of car rental? That'll be $50!
    if latest_arrival > earliest_departure: total_price += 50

    return total_price + total_wait

In [85]:
schedule_cost(s)

5285

In [86]:
def random_optimize(domain, costf):
    best = 999999999
    bestr = None

    for i in range(1000):
        # Create a random solution.
        r = [random.randint(domain[i][0], domain[i][1])
             for i in range(len(domain))]
        
        # Get the cost.
        cost = costf(r)
        
        # Compare it to the best one so far.
        if cost < best:
            best = cost
            bestr = r
    return r

In [87]:
domain = [(0,8)] * (len(people) * 2)
s = random_optimize(domain, schedule_cost)
schedule_cost(s)

5732

In [88]:
print_schedule(s)

Seymour   BOS         8:04-10:11 $ 95 15:25-16:58 $ 62
Franny    DAL        15:44-18:55 $382 10:51-14:16 $256
Zooey     CAK        10:53-13:36 $189 13:37-15:33 $142
Walt      MIA        14:01-17:24 $338 11:08-14:38 $262
Buddy     ORD        11:01-12:39 $260  7:50-10:08 $164
Les       OMA         7:39-10:24 $219  9:31-11:43 $210


In [89]:
def hill_climbing(domain, costf):
    # Create a random solution.
    sol = [random.randint(domain[i][0], domain[i][1])
           for i in range(len(domain))]
    
    # Main loop.
    while 1:
        # Create list of neighbouring solution.
        neighbors = []
        for j in range(len(domain)):
            
            # One away in each direction.
            if sol[j] > domain[j][0]:
                neighbors.append(sol[0:j]+[sol[j]+1]+sol[j+1:])
            
            if sol[j] < domain[j][1]:
                neighbors.append(sol[0:j]+[sol[j]-1]+sol[j+1:])
        
        # See what the best solution amongst the neighbors is.
        current = costf(sol)
        best = current
        for j in range(len(neighbors)):
            cost = costf(neighbors[j])
            if cost < best:
                best = cost
                sol = neighbors[j]
                
        # If there are no improvements, then we've reached the top.
        if best == current:
            break
        
    return sol

In [90]:
s = hill_climbing(domain, schedule_cost)
print_schedule(s)

Seymour   BOS        18:34-19:36 $136 13:39-15:30 $ 74
Franny    DAL        13:54-18:02 $294 17:14-20:59 $277
Zooey     CAK        17:08-19:08 $262 13:37-15:33 $142
Walt      MIA        14:01-17:24 $338 18:07-21:30 $355
Buddy     ORD        18:48-21:45 $246 17:06-20:00 $ 95
Les       OMA        16:51-19:09 $147 15:07-17:21 $129


## Simulated annealing

Optimization method inspired by physics. 

In [91]:
def annealing_optimization(domain, costf, T=10000.0, cool=0.95,step=1):
    # Initialize the values randomly.
    vec = [float(random.randint(domain[i][0], domain[i][1]))
           for i in range(len(domain))]
    
    while T > 0.1:
        # Choose one of the indices.
        i = random.randint(0, len(domain) - 1)
        
        # Choose a direction to change it.
        dir = random.randint(-step, step)
        
        # Create a new list with one of the values changed.
        vecb = vec[:]
        vecb[i] += dir
        if vecb[i] < domain[i][0]: vecb[i] = domain[i][0]
        elif vecb[i] > domain[i][1]: vecb[i] = domain[i][1]
            
        # Calculate the current cost and the new cost.
        ea = costf(vec)
        eb = costf(vecb)
        p = pow(math.e, (-eb-ea)/T)
        
        # Is it better, or does it make the probability cutoff?
        if (eb < ea or random.random() < p):
            vec = vecb
        
        # Decrease the temperature.
        T = T * cool
    return [int(v) for v in vec]

In [92]:
s = annealing_optimization(domain, schedule_cost)
print_schedule(s)

Seymour   BOS        12:34-15:02 $109 10:33-12:03 $ 74
Franny    DAL        10:30-14:57 $290 10:51-14:16 $256
Zooey     CAK        10:53-13:36 $189 10:32-13:16 $139
Walt      MIA        11:28-14:40 $248  8:23-11:07 $143
Buddy     ORD         8:25-10:34 $157  9:11-10:42 $172
Les       OMA         9:15-12:03 $ 99  8:04-10:59 $136


## Genetic algorithms

In [93]:
def genetic_optimize(domain, costf, popsize=50, step=1, mutprob=0.2, elite=0.2, maxiter=100):
        """
        popsize: the size of the population
        mutprob: the probability that a new member of the population will be a mutation rather than a crossover
        elite: the fraction of the population that are considered good solutions and are allowed to pass into the next generation
        maxiter: the number of generations to run
        """
        
        # Mutation operation.
        def mutate(vec):
            i = random.randint(0, len(domain) -1)
            if random.random() < 0.5 and vec[i] > domain[i][0]:
                return vec[0:i] + [vec[i]-step] + vec[i+1:]
            elif vec[i] < domain[i][1]:
                return vec[0:i] + [vec[i]+step] + vec[i+1:]
        
        # Crossover operation.
        def crossover(r1, r2):
            i = random.randint(1,len(domain)-2)
            return r1[0:i] + r2[i:]
        
        # Build the initial population.
        pop = []
        for i in range(popsize):
            vec = [random.randint(domain[i][0], domain[i][1])
                   for i in range(len(domain))]
            pop.append(vec)
        
        # How many winners from each generation?
        top_elite = int(elite * popsize)
        
        # Main loop.
        for i in range(maxiter):
            scores = [(costf(v), v) for v in pop]
            scores.sort()
            ranked = [v for (s, v) in scores]
            
            # Start with the pure winners.
            pop = ranked[0:top_elite]
            
            # Add mutated and bred forms of the winners.
            while len(pop) < popsize:
                if random.random() < mutprob:
                    # Mutation.
                    c = random.randint(0, top_elite)
                    pop.append(mutate(ranked[c]))
                else:
                    # Crossover.
                    c1 = random.randint(0, top_elite)
                    c2 = random.randint(0, top_elite)
                    pop.append(crossover(ranked[c1], ranked[c2]))
            # Print current best score.
            # print(scores[0][0])
        return scores[0][1]

In [94]:
s = genetic_optimize(domain, schedule_cost)
print_schedule(s)

Seymour   BOS        12:34-15:02 $109 10:33-12:03 $ 74
Franny    DAL        10:30-14:57 $290 10:51-14:16 $256
Zooey     CAK        10:53-13:36 $189 10:32-13:16 $139
Walt      MIA        11:28-14:40 $248 12:37-15:05 $170
Buddy     ORD        12:44-14:17 $134 10:33-13:11 $132
Les       OMA        11:08-13:07 $175 11:07-13:24 $171
